In [9]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [10]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [11]:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [12]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
from openai import OpenAI
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [13]:
# A challenge to test the competitive spirit

In [14]:
easy_puzzle = [
    {"role": "user", "content": 
        "You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only."},
]

In [17]:
response = ollama.chat.completions.create(model="llama3.2", messages=easy_puzzle)
display(Markdown(response.choices[0].message.content))


0.5

In [18]:
# a spicy challenge

In [19]:
dilemma_prompt = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:
Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.
Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.
If both steal, you both get nothing.
Do you choose to Steal or Share? Pick one.
"""

dilemma = [
    {"role": "user", "content": dilemma_prompt},
]


In [22]:
response = ollama.chat.completions.create(model="llama3.2", messages=dilemma)
display(Markdown(response.choices[0].message.content))


I'll choose... Share.

Since I'm playing the game with a partner, we're essentially in this together (or rather, not-so-together). We each gain something from cooperating: if both of us choose "Share", we get $1,000 each. On the other hand, defecting may result in either one of us getting nothing or $2,000.

I'd like to assume that my partner will also make the same choice (i.e., they'll choose "Share"). Not only does sharing provide a mutual benefit for both of us, but it also allows us to maintain some trust and cooperation with each other. Who knows what our individual interests are? If we're honest about our goals and intentions, we might actually work out better together.

Let's take the risk and choose to share!

In [23]:
# another hard puzzle

In [24]:
hard = """
On a bookshelf, two volumes of Pushkin stand side by side: the first and the second.
The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick.
A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume.
What distance did it gnaw through?
"""
hard_puzzle = [
    {"role": "user", "content": hard}
]

In [25]:
response = ollama.chat.completions.create(model="llama3.2", messages=hard_puzzle)
display(Markdown(response.choices[0].message.content))


To find the total distance the worm gnawed through, we need to calculate the thickness of the volumes and add it to the thickness of each cover.

Each volume has a thickness of 2 cm. Since there are two volumes, we multiply this value by 2:

2 cm/volume × 2 volumes = 4 cm

Now we add the thickness of both covers (each 2 mm thick):

4 cm + (2 mm + 2 mm) 
= 4cm+4mm 

The answer: 10/3= 4cm and 4,mm.

In [ ]:
# LangChain

In [28]:
tell_a_joke = [
    {"role": "user", "content": "Tell a joke for a student on the journey to becoming an expert in LLM Engineering"},
]

In [44]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2")
response = llm.invoke("tell_a_joke")
display(Markdown(response))


Here's one:

What do you call a fake noodle?

An impasta!

Hope that made you smile! Do you want to hear another?

In [ ]:
#LiteLLM

In [45]:
from litellm import completion
response = completion(model="ollama/llama3.2", messages=tell_a_joke)
reply = response.choices[0].message.content
display(Markdown(reply))

Why did the neural network go to therapy?

Because it was struggling with its weights and biases!

As a student on your journey to becoming an expert in LLM Engineering, remember that just like a well-calibrated model, your understanding of language and cognition should be nuanced and balanced. Keep working through the layers, and don't be afraid to retrain your perspective!

In [46]:
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Total cost: {response._hidden_params["response_cost"]*100:.4f} cents")

Input tokens: 46
Output tokens: 74
Total tokens: 120
Total cost: 0.0000 cents


In [48]:
with open("hamlet.txt", "r", encoding="utf-8") as f:
    hamlet = f.read()

loc = hamlet.find("Speak, man")
print(hamlet[loc:loc+100])

Speak, man.
  Laer. Where is my father?
  King. Dead.
  Queen. But not by him!
  King. Let him deman


In [49]:
question = [{"role": "user", "content": "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?"}]

In [51]:
response = completion(model="ollama/llama3.2", messages=question)
display(Markdown(response.choices[0].message.content))

In Hamlet, when Laertes asks "Where is my father?" his reply comes from Ophelia, who had been searching for her father Polonius's body in the castle. She tells Laertes that she hasn't found it yet.

The quote is: 

"Tell me, where he lies."